<a href="https://colab.research.google.com/github/NguyenHNhan/pyvista_3D/blob/main/M%E1%BB%99t_s%E1%BB%91_b%E1%BB%99_l%E1%BB%8Dc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#4.1 Read File

In [ ]:
mesh = pv.read(filename)
cpos = mesh.plot()

In [ ]:
mesh.points

# 4.2 Extract Cell Centers

In [ ]:
import pyvista as pv
from pyvista import examples

In [ ]:
mesh = examples.download_teapot()

In [ ]:
cpos = [
    (6.192871661244108, 5.687542355343226, -4.95345468836544),
    (0.48853358141600634, 1.2019347531215714, 0.1656178278582367),
    (-0.40642070472687936, 0.8621356761976646, 0.30256286387543047),
]

In [ ]:
centers = mesh.cell_centers()
pl = pv.Plotter()
pl.add_mesh(mesh, show_edges=True, line_width=1)
pl.add_mesh(centers, color="r", point_size=8.0, render_points_as_spheres=True)
pl.show(cpos=cpos)

In [ ]:
# Lấy các tâm của lưới có hình học 3D
grid = examples.download_letter_a()
cpos = [
    (2.704583323659036, 0.7822568412034183, 1.7251126717482546),
    (3.543391913452799, 0.31117673768140197, 0.16407006760146028),
    (0.1481171795711516, 0.96599698246102, -0.2119224645762945),
]
centers = grid.cell_centers()
pl = pv.Plotter()
pl.add_mesh(grid, show_edges=True, opacity=0.5, line_width=1)
pl.add_mesh(centers, color="r", point_size=8.0, render_points_as_spheres=True)
pl.show(cpos=cpos)

pl = pv.Plotter()
pl.add_mesh(grid.extract_all_edges(), color="k", line_width=1)
pl.add_mesh(centers, color="r", point_size=8.0, render_points_as_spheres=True)
pl.show(cpos=cpos)

# 4.3 Collision

In [ ]:
import numpy as np
import pyvista as pv
pv.set_plot_theme("document")

In [ ]:
sphere0 = pv.Sphere()
sphere0['collisions'] = np.zeros(sphere0.n_cells, dtype=bool)
# This mesh will be the moving mesh
sphere1 = pv.Sphere(radius=0.6, center=(-1, 0, 0))

In [ ]:
pl = pv.Plotter()
pl.enable_hidden_line_removal()
pl.add_mesh(sphere0, scalars='collisions', show_scalar_bar=False, cmap='bwr')
pl.camera_position = 'xz'
pl.add_mesh(sphere1, style='wireframe', color='green', line_width=5)
pl.open_gif("collision_movie.gif")

delta_x = 0.05
for _ in range(int(2 / delta_x)):
    sphere1.translate([delta_x, 0, 0], inplace=True)
    col, n_contacts = sphere0.collision(sphere1)

    collision_mask = np.zeros(sphere0.n_cells, dtype=bool)
    if n_contacts:
        collision_mask[col['ContactCells']] = True
    sphere0['collisions'] = collision_mask
    pl.write_frame()

pl.close()

# 4.4 Trích xuất biên

In [ ]:
# Download the example CAD model and extract all feature edges above 45 degrees
mesh = examples.download_cad_model()
edges = mesh.extract_feature_edges(45)

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh, color=True)
p.add_mesh(edges, color="red", line_width=5)
p.camera.zoom(1.5)
p.show()

# 4.5 Biểu diễn bề mặt - Marching Cubes
Spider Cage

In [ ]:
a = 0.9
def spider_cage(x, y, z):
    x2 = x * x
    y2 = y * y
    x2_y2 = x2 + y2
    return (np.sqrt((x2 - y2) ** 2 / x2_y2 + 3 * (z * np.sin(a)) ** 2) - 3) ** 2 + 6 * (
        np.sqrt((x * y) ** 2 / x2_y2 + (z * np.cos(a)) ** 2) - 1.5
    ) ** 2

In [ ]:
n = 100
x_min, y_min, z_min = -5, -5, -3
grid = pv.UniformGrid(
    dimensions=(n, n, n),
    spacing=(abs(x_min) / n * 2, abs(y_min) / n * 2, abs(z_min) / n * 2),
    origin=(x_min, y_min, z_min),
)
x, y, z = grid.points.T

In [ ]:
values = spider_cage(x, y, z)
mesh = grid.contour([1], values, method='marching_cubes')
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(scalars=dist, smooth_shading=True, specular=1, cmap="plasma", show_scalar_bar=False)

Barth Sextic

In [ ]:
phi = (1 + np.sqrt(5)) / 2
phi2 = phi * phi
def barth_sextic(x, y, z):
    x2 = x * x
    y2 = y * y
    z2 = z * z
    arr = (
        3 * (phi2 * x2 - y2) * (phi2 * y2 - z2) * (phi2 * z2 - x2)
        - (1 + 2 * phi) * (x2 + y2 + z2 - 1) ** 2
    )
    nan_mask = x2 + y2 + z2 > 3.1
    arr[nan_mask] = np.nan
    return arr

In [ ]:
n = 100
k = 2.0
x_min, y_min, z_min = -k, -k, -k
grid = pv.UniformGrid(
    dimensions=(n, n, n),
    spacing=(abs(x_min) / n * 2, abs(y_min) / n * 2, abs(z_min) / n * 2),
    origin=(x_min, y_min, z_min),
)
x, y, z = grid.points.T

In [ ]:
values = barth_sextic(x, y, z)
mesh = grid.contour([0], values, method='flying_edges')
dist = np.linalg.norm(mesh.points, axis=1)
mesh.plot(scalars=dist, smooth_shading=True, specular=1, cmap="plasma", show_scalar_bar=False)

Render Animate (chuyển động)

In [ ]:
def angle_to_range(angle):
    return -2 * np.sin(angle)
pl = pv.Plotter(window_size=[800, 800], off_screen=True)
pl.open_gif('barth_sextic.gif')
for angle in np.linspace(0, np.pi, 20, endpoint=False):
    pl.clear()
    pl.enable_lightkit()
    mesh = grid.contour([angle_to_range(angle)], values, method='flying_edges')
    dist = np.linalg.norm(mesh.points, axis=1)
    pl.add_mesh(
        mesh,
        scalars=dist,
        smooth_shading=True,
        specular=1,
        rng=[0.5, 1.5],
        cmap="plasma",
        show_scalar_bar=False,
    )
    pl.write_frame()
pl.close()

# 4.6 Làm mịn Gaussian

In [ ]:
data = examples.download_gourds()
cp = [(319.5, 239.5, 1053.7372980874645), (319.5, 239.5, 0.0), (0.0, 1.0, 0.0)]

In [ ]:
p = pv.Plotter(shape=(2, 2))
p.subplot(0, 0)
p.add_text("Original Image", font_size=14)
p.add_mesh(data, rgb=True)
p.camera_position = cp

p.subplot(0, 1)
p.add_text("Gaussian smoothing, std=2", font_size=14)
p.add_mesh(data.gaussian_smooth(std_dev=2.0), rgb=True)
p.camera_position = cp

p.subplot(1, 0)
p.add_text("Gaussian smoothing, std=4", font_size=14)
p.add_mesh(data.gaussian_smooth(std_dev=4.0), rgb=True)
p.camera_position = cp

p.subplot(1, 1)
p.add_text("Gaussian smoothing, std=8", font_size=14)
p.add_mesh(data.gaussian_smooth(std_dev=8.0), rgb=True)
p.camera_position = cp

p.show()

# 4.7 Biến đổi Fourier

In [ ]:
freq = [10, 5, 0]
noise = pv.perlin_noise(1, freq, (0, 0, 0))
xdim, ydim = (2**9, 2**9)
sampled = pv.sample_function(noise, bounds=(0, 10, 0, 10, 0, 10), dim=(xdim, ydim, 1))

warped_noise = sampled.warp_by_scalar()
warped_noise.plot(show_scalar_bar=False, text='Perlin Noise', lighting=False)

In [ ]:
# Triển khai FFT
sampled_fft = sampled.fft()
freq = np.fft.fftfreq(sampled.dimensions[0], sampled.spacing[0])
max_freq = freq.max()

# only show the first quadrant
subset = sampled_fft.extract_subset((0, xdim // 2, 0, ydim // 2, 0, 0))

In [ ]:
# Vẽ miền tần số
# scale to make the plot viewable
subset['scalars'] = np.abs(subset.active_scalars)
warped_subset = subset.warp_by_scalar(factor=0.0001)

pl = pv.Plotter(lighting='three lights')
pl.add_mesh(warped_subset, cmap='blues', show_scalar_bar=False)
pl.show_bounds(
    axes_ranges=(0, max_freq, 0, max_freq, 0, warped_subset.bounds[-1]),
    xtitle='X Frequency',
    ytitle='Y Frequency',
    ztitle='Amplitude',
    show_zlabels=False,
    color='k',
    font_size=26,
)
pl.add_text('Frequency Domain of the Perlin Noise')
pl.show()

In [ ]:
# Bộ lọc thông thấp
low_pass = sampled_fft.low_pass(1.0, 1.0, 1.0).rfft()
low_pass['scalars'] = np.real(low_pass.active_scalars)
warped_low_pass = low_pass.warp_by_scalar()
warped_low_pass.plot(show_scalar_bar=False, text='Low Pass of the Perlin Noise', lighting=False)

In [ ]:
# Bộ lọc thông cao
high_pass = sampled_fft.high_pass(1.0, 1.0, 1.0).rfft()
high_pass['scalars'] = np.real(high_pass.active_scalars)
warped_high_pass = high_pass.warp_by_scalar()
warped_high_pass.plot(show_scalar_bar=False, text='High Pass of the Perlin Noise', lighting=False)

Tạo Animate

In [ ]:
def warp_low_pass_noise(cfreq, scalar_ptp=sampled['scalars'].ptp()):
    """Process the sampled FFT and warp by scalars."""
    output = sampled_fft.low_pass(cfreq, cfreq, cfreq).rfft()

    # on the left: raw FFT magnitude
    output['scalars'] = output.active_scalars.real
    warped_raw = output.warp_by_scalar()

    # on the right: scale to fixed warped height
    output_scaled = output.translate((-11, 11, 0), inplace=False)
    output_scaled['scalars_warp'] = output['scalars'] / output['scalars'].ptp() * scalar_ptp
    warped_scaled = output_scaled.warp_by_scalar('scalars_warp')
    warped_scaled.active_scalars_name = 'scalars'
    # push center back to xy plane due to peaks near 0 frequency
    warped_scaled.translate((0, 0, -warped_scaled.center[-1]), inplace=True)

    return warped_raw + warped_scaled


# Initialize the plotter and plot off-screen to save the animation as a GIF.
plotter = pv.Plotter(notebook=False, off_screen=True)
plotter.open_gif("low_pass.gif", fps=8)

# add the initial mesh
init_mesh = warp_low_pass_noise(1e-2)
plotter.add_mesh(init_mesh, show_scalar_bar=False, lighting=False, n_colors=128)
plotter.camera.zoom(1.3)

for freq in np.geomspace(1e-2, 10, 25):
    plotter.clear()
    mesh = warp_low_pass_noise(freq)
    plotter.add_mesh(mesh, show_scalar_bar=False, lighting=False, n_colors=128)
    plotter.add_text(f"Cutoff Frequency: {freq:.2f}", color="black")
    plotter.write_frame()

# write the last frame a few times to "pause" the gif
for _ in range(10):
    plotter.write_frame()

plotter.close()

# 4.8 Lưới phản chiếu

In [ ]:
airplane = examples.load_airplane()

In [ ]:
airplane_reflected = airplane.reflect((0, 0, 1), point=(0, 0, -100))

In [ ]:
p = pyvista.Plotter()
p.add_mesh(airplane, show_edges=True)
p.add_mesh(airplane_reflected, show_edges=True)
p.show()

# 4.8 Rotations
Xác định góc nhìn và trục

Xác định máy ảnh và trục. Đặt gốc tọa độ thành (3,0) - (3,0) - (3,0)

In [ ]:
mesh = examples.download_cow()
mesh.points /= 1.5  # scale the mesh

camera = pv.Camera()
camera.position = (30.0, 30.0, 30.0)
camera.focal_point = (5.0, 5.0, 5.0)

axes = pv.Axes(show_actor=True, actor_scale=2.0, line_width=5)
axes.origin = (3.0, 3.0, 3.0)

In [ ]:
p = pv.Plotter()
p.add_text("Mesh", font_size=24)
p.add_actor(axes.actor)
p.camera = camera
p.add_mesh(mesh)
p.show()

Xoay quanh trục x

In [ ]:
p = pv.Plotter()
p.add_text("X-Axis Rotation", font_size=24)
p.add_actor(axes.actor)
p.camera = camera

for i in range(6):
    rot = mesh.rotate_x(60 * i, point=axes.origin, inplace=False)
    p.add_mesh(rot)

p.show()

Xoay quanh trục y

In [ ]:
p = pv.Plotter()
p.add_text("Y-Axis Rotation", font_size=24)
p.camera = camera
p.add_actor(axes.actor)
for i in range(6):
    rot = mesh.rotate_y(60 * i, point=axes.origin, inplace=False)
    p.add_mesh(rot)

p.show()

Xoay quanh trục z

In [ ]:
p = pv.Plotter()
p.add_text("Z-Axis Rotation", font_size=24)
p.camera = camera
p.add_actor(axes.actor)
for i in range(6):
    rot = mesh.rotate_z(60 * i, point=axes.origin, inplace=False)
    p.add_mesh(rot)
p.show()

Xoay quanh một vector tùy chỉnh

In [ ]:
p = pv.Plotter()
p.add_text("Custom Vector Rotation", font_size=24)
p.camera = camera
p.add_actor(axes.actor)
for i in range(6):
    rot = mesh.copy()
    rot.rotate_vector(vector=(1, 1, 1), angle=60 * i, point=axes.origin)
    p.add_mesh(rot)
p.show()

# 4.9 Subdivide Cells

In [ ]:
mesh = examples.download_bunny_coarse().triangulate()
cpos = [
    (-0.02788175062966399, 0.19293295656233056, 0.4334449972621349),
    (-0.053260899930287015, 0.08881197167521734, -9.016948161029588e-05),
    (-0.10170607813337212, 0.9686438023715356, -0.22668272496584665),
]

In [ ]:
def plot_subdivisions(mesh, a, b):
    display_args = dict(show_edges=True, color=True)
    p = pv.Plotter(shape=(3, 3))

    for i in range(3):
        p.subplot(i, 0)
        p.add_mesh(mesh, **display_args)
        p.add_text("Original Mesh")

    def row_plot(row, subfilter):
        subs = [a, b]
        for i in range(2):
            p.subplot(row, i + 1)
            p.add_mesh(mesh.subdivide(subs[i], subfilter=subfilter), **display_args)
            p.add_text(f"{subfilter} subdivision of {subs[i]}")

    row_plot(0, "linear")
    row_plot(1, "butterfly")
    row_plot(2, "loop")

    p.link_views()
    p.view_isometric()
    return p

In [ ]:
plotter = plot_subdivisions(mesh, 1, 3)
plotter.camera_position = cpos
plotter.show()

# 4.10 Làm nhẵn bề mặt

In [ ]:
# Vector to view rough edges
cpos = [-2, 5, 3]
# Load dataset
data = examples.load_uniform()
# Extract a rugged volume
vol = data.threshold_percent(30, invert=1)
vol.plot(show_edges=True, cpos=cpos, show_scalar_bar=False)

Làm mịn Laplacian:

In [ ]:
# Smooth the surface even more
smooth = surf.smooth(n_iter=100)
smooth.plot(show_edges=True, cpos=cpos, show_scalar_bar=False)

# 4.11 Tái tạo bề mặt

In [ ]:
points = pv.wrap(pv.Sphere().points)
surf = points.reconstruct_surface()
surf

In [ ]:
pl = pv.Plotter(shape=(1, 2))
pl.add_mesh(points)
pl.add_title('Point Cloud of 3D Surface')
pl.subplot(0, 1)
pl.add_mesh(surf, color=True, show_edges=True)
pl.add_title('Reconstructed Surface')
pl.show()

# 4.12 Warping by Vectors

Đầu tiên chúng ta so sánh quả cầu không cong vênh với quả cầu cong vênh.

In [ ]:
import pyvista as pv
from pyvista import examples

sphere = examples.load_sphere_vectors()
warped = sphere.warp_by_vector()

p = pv.Plotter(shape=(1, 2))
p.subplot(0, 0)
p.add_text("Before warp")
p.add_mesh(sphere, color='white')
p.subplot(0, 1)
p.add_text("After warp")
p.add_mesh(warped, color='white')
p.show()

Sau đó, chúng tôi sử dụng một số giá trị cho hệ số tỷ lệ được áp dụng cho thao tác dọc. Áp dụng hệ số cong vênh quá cao thường có thể dẫn đến kết quả không thực tế

In [ ]:
warp_factors = [0, 1.5, 3.5, 5.5]
p = pv.Plotter(shape=(2, 2))
for i in range(2):
    for j in range(2):
        idx = 2 * i + j
        p.subplot(i, j)
        p.add_mesh(sphere.warp_by_vector(factor=warp_factors[idx]))
        p.add_text(f'factor={warp_factors[idx]}')
p.show()